#### Importing Libraries

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

#### Functions for pose detection 

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_landmarks(image, results):

    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections

def draw_styled_landmarks(image, results):

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    
    return pose

#### Extracting ans storing data from video dataset

This file must be in the same folder where video dataset is stored.

In [3]:
DATA_PATH = os.path.join('Yoga_Pose_Final') 
actions = np.array(['Warrior1'])   #change the name for other label or make a function for all lavels
no_sequences = 10
sequence_length = 5 
start_folder = 6

for i in range(5):
    for action in actions: 
        for sequence in range(no_sequences):
            try: 
                os.makedirs(os.path.join(DATA_PATH, action, str(i+25), str(sequence)))
            except:
                pass
    file_name = str(i+1) + '.mp4'
    cap = cv2.VideoCapture(file_name)
    # Set mediapipe model 
    image_list = []
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for action in actions:
            for sequence in range(no_sequences):
                for frame_num in range(sequence_length):
                    ret, frame = cap.read()
                    image, results = mediapipe_detection(frame, holistic)
                    image_list.append(image)
                    draw_styled_landmarks(image, results)
                    if frame_num == 0: 
                        cv2.imshow('OpenCV Feed', image_list[frame_num])
                        cv2.waitKey(10) #frame per 0.05 sec
                    else: 
                        cv2.imshow('OpenCV Feed', image_list[frame_num])
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, action, str(i+25), str(sequence), str(frame_num))
                    np.save(npy_path, keypoints)
                    if cv2.waitKey(10) & 0xFF == ord('q'):  # Break gracefully
                        break
        cap.release()
        cv2.destroyAllWindows()